In [ ]:
import cv2
import os
from pathlib import Path

class CropLayer(object):
    def __init__(self, params, blobs):
        self.startX = 0
        self.startY = 0
        self.endX = 0
        self.endY = 0

    def getMemoryShapes(self, inputs):
        (inputShape, targetShape) = (inputs[0], inputs[1])
        (batchSize, numChannels) = (inputShape[0], inputShape[1])
        (H, W) = (targetShape[2], targetShape[3])

        self.startX = int((inputShape[3] - targetShape[3]) / 2)
        self.startY = int((inputShape[2] - targetShape[2]) / 2)
        self.endX = self.startX + W
        self.endY = self.startY + H

        return [[batchSize, numChannels, H, W]]

    def forward(self, inputs):
        return [inputs[0][:, :, self.startY:self.endY,
                self.startX:self.endX]]

# Original paper on HED: https://arxiv.org/pdf/1504.06375.pdf
# Author's github: https://github.com/s9xie/hed

def initialize():
    protoPath = "hed_trained_model/deploy.prototxt"
    modelPath = "hed/hed_pretrained_bsds.caffemodel"

    protoPath = str(Path("hed_trained_model", "deploy.prototxt"))
    modelPath = str(Path("hed_trained_model", "hed_pretrained_bsds.caffemodel"))

    net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
    cv2.dnn_registerLayer("Crop", CropLayer) # Line that causes the Kernel to crash if run twice
    return net


def get_all_file_paths(folder_path):
  all_files = []
  for root, directories, files in os.walk(folder_path):
    for filename in files:
      absolute_path = os.path.join(root, filename)
      all_files.append((absolute_path, filename))


  return all_files

In [4]:
# ONLY RUN THIS ONCE! RUNNING MULTIPLE TIMES WILL RESLT IN ERROR 
net = initialize()

# Specify the file location

Please make sure these folders exist already in the project root before executing the "Perform HED" code.
Otherwise, you'll either encounter an error, or the program won't save any of the images

In [6]:
input_folder  = str(Path('input_folder'))           # Name of the folder containing photos
output_folder = str(Path('output_folder'))          # Name of the folder where processed images are saved
                                                    # The "output_folder" has to be manually created, otherwise images won't be saved
file_list = get_all_file_paths(input_folder)             
print("Current files in the folder:", file_list)   

Current files in the folder: [('example\\IMG_5805.JPG', 'IMG_5805.JPG'), ('example\\IMG_5806.JPG', 'IMG_5806.JPG'), ('example\\IMG_5807.JPG', 'IMG_5807.JPG'), ('example\\IMG_5810.JPG', 'IMG_5810.JPG'), ('example\\IMG_5811.JPG', 'IMG_5811.JPG'), ('example\\IMG_5812.JPG', 'IMG_5812.JPG'), ('example\\IMG_5813.JPG', 'IMG_5813.JPG'), ('example\\IMG_5814.JPG', 'IMG_5814.JPG'), ('example\\IMG_5815.JPG', 'IMG_5815.JPG'), ('example\\IMG_5816.JPG', 'IMG_5816.JPG'), ('example\\IMG_5817.JPG', 'IMG_5817.JPG'), ('example\\IMG_5818.JPG', 'IMG_5818.JPG')]


# Perform HED

The following code will perform HED on all of the images specified in the folder "input_folder".

Note that this process is quite CPU / GPU intensive. If run on a low end computer, the CPU might overheat and shutdown (blue screen) if the CPU is overclocked.
By default all computers are NOT overclocked, so this is in most cases not a concern.

In [ ]:
import time

for absolute_path, filename in file_list: 

    image = cv2.imread(absolute_path)
    height, width = image.shape[:2]

    mean_input = (95, 120, 125)
    blob = cv2.dnn.blobFromImage(image, scalefactor=1, size=(width, height),
                                mean=mean_input,
                                swapRB= False, crop=False)

    net.setInput(blob)
    hed = net.forward()
    hed = hed[0,0,:,:]
    hed = (255 * hed).astype("uint8")  #rescale to 0-255

    binary_image = cv2.threshold(hed, 0, 255, cv2.THRESH_OTSU)[1]
    
    output_path = str(Path(output_folder, f"HED_{filename}"))
    print("Processing done with:" f"{output_path}")
    cv2.imwrite(f"{output_path}", binary_image)
    time.sleep(1)       # This will set a pause for 1 second 
    break